In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("diabetes.csv")

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.corr()['Outcome'].sort_values()

BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
DiabetesPedigreeFunction    0.173844
Pregnancies                 0.221898
Age                         0.238356
BMI                         0.292695
Glucose                     0.466581
Outcome                     1.000000
Name: Outcome, dtype: float64

In [5]:
X = df.drop(columns='Outcome',axis=1)
y = df.iloc[:,-1]

In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
sc = StandardScaler()
X = sc.fit_transform(X)

In [8]:
X.shape

(768, 8)

In [9]:
from sklearn.model_selection import train_test_split
X_train , X_test, y_train , y_test = train_test_split(X,y,test_size = 0.2,random_state=1)

In [10]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [11]:
model = Sequential()

model.add(Dense(32,activation='relu',input_dim = 8))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [12]:
model.fit(X_train,y_train,batch_size=32,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100


20/20 [==============================] - 1s 10ms/step - loss: 0.6135 - accuracy: 0.7150 - val_loss: 0.5841 - val_accuracy: 0.7273
Epoch 2/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5710 - accuracy: 0.7345 - val_loss: 0.5500 - val_accuracy: 0.7662
Epoch 3/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5412 - accuracy: 0.7476 - val_loss: 0.5283 - val_accuracy: 0.7468
Epoch 4/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5216 - accuracy: 0.7573 - val_loss: 0.5134 - val_accuracy: 0.7727
Epoch 5/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5076 - accuracy: 0.7671 - val_loss: 0.5036 - val_accuracy: 0.7792
Epoch 6/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4962 - accuracy: 0.7638 - val_loss: 0.4957 - val_accuracy: 0.7792
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4879 - accuracy: 0.7655 - val_loss: 0.4892 - val_accuracy: 0.7857
Epoch 8/10

# 1. How to select appropriate Optimizer
# 2. No. of Neurons in a layer
# 3. How to select no. of Hidden layer
# 4. All in One Model

In [13]:
import keras_tuner as kt

# 1. How to select appropriate Optimizer

In [14]:


def build_model(hp):
    model = Sequential()
    model.add(Dense(32,activation='relu',input_dim = 8))
    model.add(Dense(1,activation='sigmoid'))

    optimizer = hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adadelta'])
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])

    return model

In [15]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5)

Reloading Tuner from .\untitled_project\tuner0.json


In [16]:
tuner.search(X_train,y_train,epochs = 5 , validation_data = (X_test,y_test))

In [17]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [18]:
model = tuner.get_best_models(num_models=1)[0]

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
model.fit(X_train,y_train,epochs=100,batch_size=32,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 8ms/step - loss: 0.5028 - accuracy: 0.7557 - val_loss: 0.4842 - val_accuracy: 0.8052
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4942 - accuracy: 0.7622 - val_loss: 0.4768 - val_accuracy: 0.7987
Epoch 9/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4862 - accuracy: 0.7655 - val_loss: 0.4712 - val_accuracy: 0.8052
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4800 - accuracy: 0.7671 - val_loss: 0.4674 - val_accuracy: 0.8052
Epoch 11/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4754 - accuracy: 0.7655 - val_loss: 0.4671 - val_accuracy: 0.7987
Epoch 12/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4714 - accuracy: 0.7704 - val_loss: 0.4650 - val_accuracy: 0.8052
Epoch 13/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4679 - accuracy: 0.7769 - val_loss: 0.4626 - val_accuracy: 0.8117
Epoch 14/

# 2. No. of Neurons in a layer

In [21]:
def build_model(hp):

    model = Sequential()

    units = hp.Int('units' , min_value = 8,max_value = 128,step = 8)
    model.add(Dense(units=units , activation='relu',input_dim = 8))
    model.add(Dense(units=1,activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

    return model

In [22]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory = 'best_neuron_no')

Reloading Tuner from best_neuron_no\untitled_project\tuner0.json


In [23]:
tuner.search(X_train,y_train,epochs = 5,validation_data = (X_test,y_test))

In [24]:
tuner.get_best_hyperparameters()[0].values

{'units': 128}

In [25]:
model = tuner.get_best_models(num_models=1)[0]
model.fit(X_train,y_train,epochs=100,batch_size=32,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 9ms/step - loss: 0.4919 - accuracy: 0.7638 - val_loss: 0.4741 - val_accuracy: 0.7922
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4775 - accuracy: 0.7736 - val_loss: 0.4628 - val_accuracy: 0.7922
Epoch 9/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4679 - accuracy: 0.7752 - val_loss: 0.4594 - val_accuracy: 0.7922
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4605 - accuracy: 0.7736 - val_loss: 0.4568 - val_accuracy: 0.7857
Epoch 11/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4559 - accuracy: 0.7769 - val_loss: 0.4562 - val_accuracy: 0.7922
Epoch 12/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4522 - accuracy: 0.7752 - val_loss: 0.4540 - val_accuracy: 0.7857
Epoch 13/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4495 - accuracy: 0.7752 - val_loss: 0.4557 - val_accuracy: 0.7922
Epoch 14/

# 3. How to select no. of Hidden layer

In [26]:
def build_model(hp):
    model = Sequential()

    model.add(Dense(units=72,activation='relu',input_dim=8))

    for i in range(hp.Int("num_layer",min_value=1,max_value=10)):
        model.add(Dense(units=72,activation='relu'))

    model.add(Dense(units=1,activation='sigmoid'))

    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

    return model


In [27]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory = 'num_layer')



Reloading Tuner from num_layer\untitled_project\tuner0.json


In [28]:
tuner.search(X_train,y_train,epochs = 5 , validation_data = (X_test,y_test))

In [29]:
tuner.get_best_hyperparameters()[0].values

{'num_layer': 10}

In [30]:
model = tuner.get_best_models(num_models=1)[0]
model.fit(X_train,y_train,epochs=100,batch_size=32,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 3s 12ms/step - loss: 0.4439 - accuracy: 0.7883 - val_loss: 0.5442 - val_accuracy: 0.7532
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4340 - accuracy: 0.8062 - val_loss: 0.5262 - val_accuracy: 0.7532
Epoch 9/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4230 - accuracy: 0.7997 - val_loss: 0.4940 - val_accuracy: 0.7662
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4130 - accuracy: 0.8078 - val_loss: 0.5178 - val_accuracy: 0.7403
Epoch 11/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4038 - accuracy: 0.8094 - val_loss: 0.4921 - val_accuracy: 0.7468
Epoch 12/100
20/20 [==============================] - 0s 3ms/step - loss: 0.3927 - accuracy: 0.8094 - val_loss: 0.5537 - val_accuracy: 0.7468
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.3743 - accuracy: 0.8436 - val_loss: 0.5027 - val_accuracy: 0.7532
Epoch 14

# 4. All in One Model

In [31]:
def build_model(hp):

    model = Sequential()

    counter = 0

    for i in range(hp.Int("num_layers",min_value = 1,max_value = 10)):

        if counter == 0 :
            model.add(Dense(
                hp.Int('units' + str(i),min_value = 8,max_value = 128,step = 8),
                activation= hp.Choice('activation' + str(i),values = ['relu','tanh','sigmoid']),
                input_dim = 8
                
                )
                )
            model.add(Dropout(hp.Choice("Dropout" + str(i),values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        else :

                model.add(Dense(
                hp.Int('units' + str(i),min_value = 8,max_value = 128,step = 8),
                activation= hp.Choice('activation' + str(i),values = ['relu','tanh','sigmoid']),
                )
                )
                model.add(Dropout(hp.Choice("Dropout" + str(i),values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

        counter += 1

        model.add(Dense(1,activation = 'sigmoid'))

        model.compile(optimizer=hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adadelta','nadam']),
                      loss = 'binary_crossentropy',metrics=['accuracy'])
        

        return model



In [32]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory = 'Hyperparamter_check',project_name = 'Final')



In [33]:
tuner.search(X_train,y_train,epochs = 5 , validation_data = (X_test,y_test))

Trial 3 Complete [00h 00m 01s]
val_accuracy: 0.7402597665786743

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 04s


In [34]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units0': 24,
 'activation0': 'tanh',
 'Dropout0': 0.1,
 'optimizer': 'rmsprop'}

In [35]:
model = tuner.get_best_models(num_models=1)[0]
model.fit(X_train,y_train,epochs=200,initial_epoch=5,validation_data=(X_test,y_test))

Epoch 6/200
20/20 [==============================] - 1s 8ms/step - loss: 0.5305 - accuracy: 0.7557 - val_loss: 0.5168 - val_accuracy: 0.7922
Epoch 7/200
20/20 [==============================] - 0s 3ms/step - loss: 0.5191 - accuracy: 0.7524 - val_loss: 0.5033 - val_accuracy: 0.7922
Epoch 8/200
20/20 [==============================] - 0s 4ms/step - loss: 0.5100 - accuracy: 0.7590 - val_loss: 0.4925 - val_accuracy: 0.7922
Epoch 9/200
20/20 [==============================] - 0s 4ms/step - loss: 0.5048 - accuracy: 0.7476 - val_loss: 0.4853 - val_accuracy: 0.7922
Epoch 10/200
20/20 [==============================] - 0s 3ms/step - loss: 0.4893 - accuracy: 0.7655 - val_loss: 0.4814 - val_accuracy: 0.7922
Epoch 11/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4853 - accuracy: 0.7622 - val_loss: 0.4773 - val_accuracy: 0.7792
Epoch 12/200
20/20 [==============================] - 0s 3ms/step - loss: 0.4802 - accuracy: 0.7687 - val_loss: 0.4746 - val_accuracy: 0.7727
Epoch 13/2